In [ ]:
#!unzip /content/cognate_words-dj-kostya-develop.zip
#!unzip /content/mental_neighbours-main.zip
!unzip /content/cognate_words-issc.zip

Archive:  /content/cognate_words-issc.zip
e942ed7bb1dc9f92b04da8ff4fa5442bd0feb0ef
   creating: cognate_words-issc/
  inflating: cognate_words-issc/.DS_Store  
 extracting: cognate_words-issc/.gitignore  
  inflating: cognate_words-issc/README.md  
   creating: cognate_words-issc/baseline/
 extracting: cognate_words-issc/baseline/.gitignore  
  inflating: cognate_words-issc/baseline/baseline.py  
   creating: cognate_words-issc/baseline/config/
  inflating: cognate_words-issc/baseline/config/morph_config_1-data.json  
  inflating: cognate_words-issc/baseline/config/morph_config_3-5-192-1.json  
  inflating: cognate_words-issc/baseline/config/morph_config_3-5-192-3-memo_dima.json  
  inflating: cognate_words-issc/baseline/config/morph_config_3-5-192-3-memo_reformat.json  
   creating: cognate_words-issc/baseline/data/
  inflating: cognate_words-issc/baseline/data/ds_common_words_utf_8.csv  
  inflating: cognate_words-issc/baseline/data/ds_common_words_utf_8_test.csv  
  inflating: cogna

In [ ]:
!pip install --upgrade openpyxl

Requirement already up-to-date: openpyxl in /usr/local/lib/python3.7/dist-packages (3.0.7)


In [ ]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in /usr/local/lib/python3.7/dist-packages (1.2.4)


In [ ]:
import sys
sys.path.append("/content/mental_neighbours-main/morpheme_evaluation")
sys.path.append("/content/cognate_words-dj-kostya-develop/baseline")
sys.path.append("/content/cognate_words-dj-kostya-develop/baseline/data/known_affixes")
sys.path.append("/content/cognate_words-issc")

In [ ]:
!pip install pqdm

In [ ]:
from neural_morph_segm import load_cls
from sklearn.metrics import f1_score
import sys
import numpy as np
import pqdm
from evristic import find_root

def getRoot(word, model):
    ans = []
    for word, (labels, probs) in zip([word], model._predict_probs([word])):
        morphemes, morpheme_probs, morpheme_types = model.labels_to_morphemes(
            word, labels, probs, return_probs=True, return_types=True)

        ans.append([(morphem, prob) for morphem, prob, morhem_type in zip(
            morphemes, morpheme_probs, morpheme_types) if morhem_type == 'ROOT'])

    return ans


def getBestRoot(res):
    return max(res, key=lambda x: x[1])


def getSubstring(str1, str2):
    from difflib import SequenceMatcher
    match = SequenceMatcher(None, str1, str2).find_longest_match(
        0, len(str1), 0, len(str2))
    return str1[match.a: match.a + match.size]


def getInput(word1, word2):
    word1, word2 = word1.lower(), word2.lower()
    ans = getRoots([word1, word2])

    roots = [(a1[0], a2[0]) for a1 in ans[0] for a2 in ans[1]]

    res = []
    
    for r in roots:
        substr = getSubstring(*r)
        if len(r) == 1:
            print(roots, word1, word2)
            continue
        if not substr:
            res.append({
                "prefix_1": r[0],
                "postfix_1": "",
                "prefix_2": r[1],
                "postfix_2": "",
                "substr_len": len(substr)
            })
            continue
        roots_splited = [rot.split(substr) for rot in r]
        res.append({
            "prefix_1": roots_splited[0][0],
            "postfix_1": roots_splited[0][1],
            "prefix_2": roots_splited[1][0],
            "postfix_2": roots_splited[1][1],
            "substr_len": len(substr)
        })
    return res


res = None

eng_titles = {
    "prefix_1": 3,
    "prefix_2": 4,
    "postfix_1": 5,
    "postfix_2": 6,
    "substr_len": 7
}

for_insert = []


def parseRow(row):
    a = getInput(row[0], row[1])
    if row[-1] == 1 and len(a) > 1:
        row[-1] = 2
    for key in a[0]:
        row[eng_titles[key]] = a[0][key]

    for idx, _ in enumerate(a[1:]):
        a[idx]['Корень_x'] = row[0]
        a[idx]['Корень_y'] = row[1]
        a[idx]['key'] = row[-1]
    
    for_insert.extend(a[1:])
    return row

# def prepaire_root(row):
#     root_split = row[1].split('_')[0].split('(')
#     base_root = root_split[0]
#     if len(root_split) > 1:
#         end_root = root_split[1][:-1]
#         other_root = f'{base_root[:-len(end_root)]}{end_root}'
#         if other_root in row[0]:
#             return (row[0], other_root)
#     return (row[0], base_root)
def prepaire_root(row):
    root_split = row['root'].split()
    return generate_bitmask_for_list(row['word'],root_split)

def generate_bitmask(word: str, root: str):
    bitmask = [0] * len(word)
    start_idx = word.find(root)
    for i in range(start_idx, start_idx+len(root)):
        bitmask[i] = 1
    return bitmask

def generate_bitmask_for_list(word: str, roots: list):
    bitmask = [0] * len(word)
    for r in roots:
        start_idx = word.find(r)
        for i in range(start_idx, start_idx+len(r)):
            bitmask[i] = 1
    return bitmask

def f1_for_row(row):
    return f1_score(row[1], row[2])

def get_only_root(pairs):
    return [p[0] for p in pairs]


In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, precision_recall_fscore_support

model = load_cls("/content/cognate_words-dj-kostya-develop/baseline/models/morphemes-3-5-3-memo_reformat.json")

172
77
17459
4050
18981
4137
357
109
35
15
16
12
17639
4079
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
symbol_inputs (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
context_inputs (InputLayer)     [(None, None, 15)]   0                                            
__________________________________________________________________________________________________
symbol_embeddings (Lambda)      (None, None, 37)     0           symbol_inputs[0][0]              
__________________________________________________________________________________________________
dropout (Dropout)               (None, None, 15)     0           context_inputs[0][0]             
__________________________________

In [ ]:
df = pd.read_excel("/content/ds_common_words_utf_8.xlsx")

In [ ]:
df

,Слово,Корень
0,международный,народ
1,научный,уч
2,студенческий,студенч
3,конференция,конференци
4,самолёт,сам
...,...,...
1030,крашеный,крас_3
1031,окрас,крас_3
1032,подкрасить,крас_3
1033,раскраска,крас_3


In [ ]:
df.columns = ["word", "root_group"]

In [ ]:
df['roots'] = df['word'].apply(getRoot, args=(model,))

Processing morphemes for bucket length 15
Processing morphemes for bucket length 15 finished
Bucket 1 predicting
Processing morphemes for bucket length 9
Processing morphemes for bucket length 9 finished
Bucket 1 predicting
Processing morphemes for bucket length 14
Processing morphemes for bucket length 14 finished
Bucket 1 predicting
Processing morphemes for bucket length 14
Processing morphemes for bucket length 14 finished
Bucket 1 predicting
Processing morphemes for bucket length 9
Processing morphemes for bucket length 9 finished
Bucket 1 predicting
Processing morphemes for bucket length 11
Processing morphemes for bucket length 11 finished
Bucket 1 predicting
Processing morphemes for bucket length 11
Processing morphemes for bucket length 11 finished
Bucket 1 predicting
Processing morphemes for bucket length 12
Processing morphemes for bucket length 12 finished
Bucket 1 predicting
Processing morphemes for bucket length 13
Processing morphemes for bucket length 13 finished
Bucket 

In [ ]:
roots_list = df['roots'].tolist()
words = df['word'].tolist()

In [ ]:
word2roots = dict()

for word, roots in zip(words, roots_list):
    word2roots[word] = list()
    for root, prob in roots[0]:
        word2roots[word].append(root)

In [ ]:
df = df.drop(columns=['roots'])

In [ ]:
df

,word,root_group
0,международный,народ
1,научный,уч
2,студенческий,студенч
3,конференция,конференци
4,самолёт,сам
...,...,...
1030,крашеный,крас_3
1031,окрас,крас_3
1032,подкрасить,крас_3
1033,раскраска,крас_3


In [ ]:
df = df.merge(df, how="cross")

In [ ]:
df.columns = ['word_1', 'root_group_1', 'word_2', 'root_group_2']

In [ ]:
df

,word_1,root_group_1,word_2,root_group_2
0,международный,народ,международный,народ
1,международный,народ,научный,уч
2,международный,народ,студенческий,студенч
3,международный,народ,конференция,конференци
4,международный,народ,самолёт,сам
...,...,...,...,...
1071220,раскрашивание,крас_3,крашеный,крас_3
1071221,раскрашивание,крас_3,окрас,крас_3
1071222,раскрашивание,крас_3,подкрасить,крас_3
1071223,раскрашивание,крас_3,раскраска,крас_3


In [ ]:
df['cognate'] = df['root_group_1'] == df['root_group_2']

In [ ]:
def equal_roots(word_1, word_2):
    roots_1 = set(word2roots[word_1])
    roots_2 = set(word2roots[word_2])
    return bool(roots_1.intersection(roots_2))

In [ ]:
df['equal_roots'] = df.apply(lambda x: equal_roots(x.word_1, x.word_2), axis=1)

In [ ]:
precision_recall_fscore_support(df['equal_roots'].tolist(), df['cognate'].tolist(), average='binary')

(0.6724286272976144, 0.8383227693807899, 0.7462673611111111, None)

In [ ]:
getRoot(word='белошвейка', model=model)

Processing morphemes for bucket length 12
Processing morphemes for bucket length 12 finished
Bucket 1 predicting


[[('бел', 0.9915142200619572), ('шв', 0.9493664785267448)]]

# С чередованием

In [ ]:
df

,word_1,root_group_1,word_2,root_group_2,cognate,equal_roots
0,международный,народ,международный,народ,True,True
1,международный,народ,научный,уч,False,False
2,международный,народ,студенческий,студенч,False,False
3,международный,народ,конференция,конференци,False,False
4,международный,народ,самолёт,сам,False,False
...,...,...,...,...,...,...
1071220,раскрашивание,крас_3,крашеный,крас_3,True,True
1071221,раскрашивание,крас_3,окрас,крас_3,True,False
1071222,раскрашивание,крас_3,подкрасить,крас_3,True,False
1071223,раскрашивание,крас_3,раскраска,крас_3,True,False


In [ ]:
import root_alternation
root_alternation.find_possible_root_alternations("гар")

['гар',
 'гарy',
 'гарe',
 'гeар',
 'гарoй',
 'гарa',
 'гары',
 'гарo',
 'гoар',
 'гаeр',
 'гиар',
 'гаир',
 'гаoр',
 'гари']

In [ ]:
mix_roots = {"гар":"гор","клан":"клон", "твар":"твор","зар":"зор","плав":"плов", "кас":"кос","мак":"мок","равн":"ровн","раст":"ращ","ращ":"рос","раст":"рос","скак":"скоч","лаг":"лож",
             "бер":"бир","дер":"дир","мер":"мир","пер":"пир","тер":"тир","жег":"жиг","блест":"блист","стел":"стил","чет":"чит"}
tmp = {}
for i in mix_roots:
  tmp[mix_roots[i]] = i
mix_roots.update(tmp)

In [ ]:
import numpy as np
def mixxing(root1,root2):
  root1_mix = root_alternation.find_possible_root_alternations(root1)
  root2_mix = root_alternation.find_possible_root_alternations(root2)
  if root1 in mix_roots:
    root1_mix.append(mix_roots[root1])
    root2_mix.append('')
  if root2 in mix_roots:
    root1_mix.append('')
    root2_mix.append(mix_roots[root2])
  root1_array = np.array(root1_mix)
  root2_array = np.array(root2_mix)

  if len(list(set(root1_mix).intersection(root2_mix))) == 0:
    return False
  else:
    return True

print(mixxing('гар','гор'))
print(mixxing('гар','бел'))

True
False


In [ ]:
p = df.apply(lambda x: mixxing(x['root_group_1'], x['root_group_2a']), axis=1)

In [ ]:
precision_recall_fscore_support(p.tolist(), df['cognate'].tolist(), average='binary')

(1.0, 0.9723172864856643, 0.9859643710958587, None)

# BERT

In [ ]:
!pip install bert-score

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 2.1MB 7.6MB/s 
     |████████████████████████████████| 870kB 25.1MB/s 
     |████████████████████████████████| 3.3MB 35.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d85d77fec9b47886caade6962218f95eda11e835da897e10ed9c1265cd525df1
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import bert_score

In [ ]:
scorer = bert_score.BERTScorer(lang='ru')

In [ ]:
p_words = df.apply(lambda x: scorer.score([x['root_group_1']], [x['root_group_2']])[2], axis=1)